In [ ]:
import numpy as np
import random
import numpy as np

def createInputLayer(j):
    wlc = np.round(np.arange(0.5, 1.5, 0.05), 2)
    wleaks = np.random.choice(wlc, size=j+1, replace=True)

    netlist = ".subckt layer_input vdd vss vin "
    for i in range(1, j+1):
        netlist+=f"vout{i} "
    netlist += "\n"

    for i in range(1, j+1):
        device = f"Xx{i} net{i} vdd vg100n vout{i} vileak vss ul_tun W_LEAK={wleaks[i]} \n"
        netlist += device
    
    types_curr_sc = np.array(["syn_pos", "syn_neg"])
    curr_sc = np.random.choice(types_curr_sc, size=j+1, replace=True)

    for i in range(1, j+1):
        dev_curr_scs = f"Xcs{i} net{i} vdd vin vout{i} vss {curr_sc[i]} \n"
        netlist += dev_curr_scs

    netlist += """I9 vg100n vss 100nA
XM4 vg100n vg100n vdd vdd sky130_fd_pr__pfet_01v8 L=0.15 W=2 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29'
+ pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM2 vileak vileak vss vss sky130_fd_pr__nfet_01v8 L=0.15 W=10 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29'
+ pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
I1 vdd vileak 100nA\n"""

    netlist += ".ends \n"
    print(netlist)


def createHiddenLayer(j):
    wlc = np.round(np.arange(0.5, 1.5, 0.05), 2)
    wleaks = np.random.choice(wlc, size=j+1, replace=True)

    netlist = ".subckt layer_hidden vdd vss"
    for i in range(1, j+1):
        netlist+=f"Iext{i} "
    
    for i in range(1, j+1):
        netlist+=f"vout{i} "

    netlist += "\n"

    for i in range(1, j+1):
        device = f"Xx{i} Iext{i} vdd vg100n vout{i} vileak vss ul_tun W_LEAK={wleaks[i]} \n"
        netlist += device

    netlist += """I9 vg100n vss 100nA
XM4 vg100n vg100n vdd vdd sky130_fd_pr__pfet_01v8 L=0.15 W=2 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29'
+ pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM2 vileak vileak vss vss sky130_fd_pr__nfet_01v8 L=0.15 W=10 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29'
+ pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
I1 vdd vileak 100nA\n"""

    netlist += ".ends \n"
    print(netlist)

def createOutputLayer(j):
    wlc = np.round(np.arange(0.5, 1.5, 0.05), 2)
    wleaks = np.random.choice(wlc, size=j+1, replace=True)

    netlist = ".subckt layer_output_rew vdd vss Vreward \n"
    for i in range(1, j+1):
        netlist+=f"Iext{i} "

    for i in range(1, j+1):
        netlist+=f"vout{i} "

    netlist += "\n"

    for i in range(1, j+1):
        device = f"Xx{i} Iext{i} vdd vg100n vout{i} Vreward vss ul_tun W_LEAK={wleaks[i]} \n"
        netlist += device

    for i in range(1, j+1):
        transistor = f"""XM{i} net1 vout{i} GND GND sky130_fd_pr__nfet_01v8 L=0.15 W=5 nf=1 ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1 \n"""
        netlist += transistor
   
    netlist += "x14 VDD Vout net1 GND integrator\n"

    netlist += ".ends \n"
    print(netlist)

def createSynapseArray(z, w):
    netlist = f".subckt stdp_{z}x{w} vdda vsss "

    for i in range(1, z+1):
        netlist+=f"vout_pre{i} "

        for i in range(1, w+1):
        netlist += f"i_post{i} "

    for i in range(1, w+1):
        netlist+=f"vout_post{i} "

    
    
    netlist += "\n"

    for i in range(1, z+1):
        for j in range(1, w+1):
            netlist +=f"xstdp{i}{j} vdda vssa vout_pre{i} vout_post{j} i_post{j} stdp \n"

    netlist += ".ends \n"
    print(netlist)



In [72]:
createSynapseArray(8, 4)

.subckt layer_input vdd vss vout_pre1 vout_pre2 vout_pre3 vout_pre4 vout_pre5 vout_pre6 vout_pre7 vout_pre8 vout_post1 vout_post2 vout_post3 vout_post4 i_post1 i_post2 i_post3 i_post4 
xstdp11 vdda vssa vout_pre1 vout_post1 i_post1 stdp 
xstdp12 vdda vssa vout_pre1 vout_post2 i_post2 stdp 
xstdp13 vdda vssa vout_pre1 vout_post3 i_post3 stdp 
xstdp14 vdda vssa vout_pre1 vout_post4 i_post4 stdp 
xstdp21 vdda vssa vout_pre2 vout_post1 i_post1 stdp 
xstdp22 vdda vssa vout_pre2 vout_post2 i_post2 stdp 
xstdp23 vdda vssa vout_pre2 vout_post3 i_post3 stdp 
xstdp24 vdda vssa vout_pre2 vout_post4 i_post4 stdp 
xstdp31 vdda vssa vout_pre3 vout_post1 i_post1 stdp 
xstdp32 vdda vssa vout_pre3 vout_post2 i_post2 stdp 
xstdp33 vdda vssa vout_pre3 vout_post3 i_post3 stdp 
xstdp34 vdda vssa vout_pre3 vout_post4 i_post4 stdp 
xstdp41 vdda vssa vout_pre4 vout_post1 i_post1 stdp 
xstdp42 vdda vssa vout_pre4 vout_post2 i_post2 stdp 
xstdp43 vdda vssa vout_pre4 vout_post3 i_post3 stdp 
xstdp44 vdda vssa vo